In [1]:
import os
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

from torch.cuda.amp import GradScaler, autocast
from torchvision import transforms, models
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'


In [2]:
train_img_dir = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\train\images"
val_img_dir = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\val\images"


In [3]:
df = pd.read_csv(r'C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\train\fairface_label_train_cleaned.csv')
df


,file,age,gender,race,service_test
0,1.jpg,50-59,Male,East Asian,True
1,2.jpg,30-39,Female,Indian,False
2,3.jpg,3-9,Female,Black,False
3,4.jpg,20-29,Female,Indian,True
4,5.jpg,20-29,Female,Indian,True
...,...,...,...,...,...
85897,86739.jpg,20-29,Male,Middle Eastern,True
85898,86740.jpg,20-29,Male,Indian,True
85899,86741.jpg,10-19,Male,Indian,True
85900,86743.jpg,10-19,Female,Black,True


In [4]:
df = df[~df['age'].isin(['0-2', '3-9', '60-69'])]
df['race'] = df['race'].replace({'Southeast Asian': 'East Asian'})


C:\Users\shrit\AppData\Local\Temp\ipykernel_45396\913567728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].replace({'Southeast Asian': 'East Asian'})


In [5]:
age_mapping = {
    '10-19': 0,
    '20-29': 1,
    '30-39': 2,
    '40-49': 3,
    '50-59': 4,
}

df['age'] = df['age'].map(age_mapping).astype(int)

race_mapping = {
    'East Asian': 0,
    'Indian': 1,
    'Black': 2,
    'White': 3,
    'Middle Eastern': 4,
    'Latino_Hispanic': 5
}

#numerical labels
df['race'] = df['race'].map(race_mapping)


gender_mapping = {
    'Male': 0,
    'Female': 1
}

#  binary 
df['gender'] = df['gender'].map(gender_mapping)

train_df = df.drop(columns=['service_test'])

C:\Users\shrit\AppData\Local\Temp\ipykernel_45396\3960336617.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['age'].map(age_mapping).astype(int)
C:\Users\shrit\AppData\Local\Temp\ipykernel_45396\3960336617.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].map(race_mapping)
C:\Users\shrit\AppData\Local\Temp\ipykernel_45396\3960336617.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [6]:
train_df.head()

,file,age,gender,race
0,1.jpg,4,0,0
1,2.jpg,2,1,1
3,4.jpg,1,1,1
4,5.jpg,1,1,1
5,6.jpg,1,0,3


In [7]:
train_df.to_csv(r'C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\train\fairface_label_train_final.csv', index=False)

In [8]:
df = pd.read_csv(r'C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\val\fairface_label_val_cleaned.csv')
df

,file,age,gender,race,service_test
0,1.jpg,3-9,Male,East Asian,False
1,2.jpg,50-59,Female,East Asian,True
2,3.jpg,30-39,Male,White,True
3,4.jpg,20-29,Female,Latino_Hispanic,True
4,5.jpg,20-29,Male,Southeast Asian,False
...,...,...,...,...,...
10831,10950.jpg,30-39,Male,White,True
10832,10951.jpg,50-59,Male,White,False
10833,10952.jpg,60-69,Male,Latino_Hispanic,False
10834,10953.jpg,20-29,Female,East Asian,False


In [9]:
df = df[~df['age'].isin(['0-2', '3-9', '60-69'])]
df['race'] = df['race'].replace({'Southeast Asian': 'East Asian'})


C:\Users\shrit\AppData\Local\Temp\ipykernel_45396\913567728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].replace({'Southeast Asian': 'East Asian'})


In [10]:
age_mapping = {
    '10-19': 0,
    '20-29': 1,
    '30-39': 2,
    '40-49': 3,
    '50-59': 4,
}


df['age'] = df['age'].map(age_mapping).astype(int)

race_mapping = {
    'East Asian': 0,
    'Indian': 1,
    'Black': 2,
    'White': 3,
    'Middle Eastern': 4,
    'Latino_Hispanic': 5
}

df['race'] = df['race'].map(race_mapping)


gender_mapping = {
    'Male': 0,
    'Female': 1
}

df['gender'] = df['gender'].map(gender_mapping)

val_df = df.drop(columns=['service_test'])



C:\Users\shrit\AppData\Local\Temp\ipykernel_45396\2122856892.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['age'].map(age_mapping).astype(int)
C:\Users\shrit\AppData\Local\Temp\ipykernel_45396\2122856892.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].map(race_mapping)
C:\Users\shrit\AppData\Local\Temp\ipykernel_45396\2122856892.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [11]:
val_df.head()

,file,age,gender,race
1,2.jpg,4,1,0
2,3.jpg,2,0,3
3,4.jpg,1,1,5
4,5.jpg,1,0,0
5,6.jpg,2,0,5


In [12]:
val_df.to_csv(r'C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\val\fairface_label_val_final.csv', index=False)

In [13]:
train_df['gender'] = train_df['gender'].astype(float)
val_df['gender'] = val_df['gender'].astype(float)

In [14]:
train_image_path = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\train\images"
val_image_path = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\fair_face\val\images"

In [15]:
class MultiTaskModel(nn.Module):
    def __init__(self, device="cuda"):
        super(MultiTaskModel, self).__init__()
        #  pretrained EfficientNet as transfer learning
        self.backbone = models.efficientnet_b0(pretrained=True)  # Use B0 variant bc its fast
        self.backbone.classifier = nn.Identity()  # remove classification layer
        
        # add custom heads for multi-task 
        self.age_head = nn.Sequential(
    nn.Dropout(0.2),  # dropout
    nn.Linear(1280, 5)
    )
        self.gender_head = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(1280, 1)
    )
        self.race_head = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(1280, 6)
    )

    def forward(self, x):
        features = self.backbone(x)  # features from the EfficientNet 
        age_out = self.age_head(features)
        gender_out = self.gender_head(features)
        race_out = self.race_head(features)
        return age_out, gender_out, race_out


In [16]:
class FairFaceDataset(Dataset): #transforming the dataset to make it work
    def __init__(self, dataframe, root_dir, transform=None):
        self.data = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        
        # Extract labels
        age = self.data.iloc[idx, 1]
        gender = self.data.iloc[idx, 2]
        race = self.data.iloc[idx, 3]

        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(age, dtype=torch.long), torch.tensor(gender, dtype=torch.float), torch.tensor(race, dtype=torch.long)


In [17]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((224, 224)),  #  EfficientNet input size
    transforms.RandomHorizontalFlip(), # augmentations
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [18]:
train_dataset = FairFaceDataset(dataframe=train_df, root_dir=train_image_path, transform=train_transform)
val_dataset = FairFaceDataset(dataframe=val_df, root_dir=val_image_path, transform=val_transform)

In [19]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False,pin_memory=True)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiTaskModel(device=device).to(device) #useing GPU

c:\Users\shrit\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\shrit\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
criterion_age = nn.CrossEntropyLoss()
criterion_gender = nn.BCEWithLogitsLoss()  # Use raw 
criterion_race = nn.CrossEntropyLoss()

In [22]:
print("Unique age labels:", train_df['age'].unique())
print("Unique race labels:", train_df['race'].unique()) # testing age labels

Unique age labels: [4 2 1 3 0]
Unique race labels: [0 1 3 4 5 2]


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MultiTaskModel(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (sca

In [24]:
optimizer = torch.optim.Adam([
    {'params': model.backbone.parameters(), 'lr': 1e-6},  #  LR for pre-trained 
    {'params': model.age_head.parameters(), 'lr': 1e-4},  #  LR for heads
    {'params': model.gender_head.parameters(), 'lr': 1e-4},
    {'params': model.race_head.parameters(), 'lr': 1e-4},
])

In [25]:
alpha, beta, gamma = 1.5, 1.0, 1.5 #loss weight multipliers

In [26]:
def train_one_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    correct_age, correct_gender, correct_race = 0, 0, 0
    total_samples = 0

    scaler = torch.amp.GradScaler()
    progress_bar = tqdm(dataloader, desc="Training", unit="batch")

    for images, age_labels, gender_labels, race_labels in progress_bar:
        images, age_labels, gender_labels, race_labels = (
            images.to(device),
            age_labels.to(device),
            gender_labels.to(device),
            race_labels.to(device),
        )

        optimizer.zero_grad()

        # mixed precision operations
        with torch.amp.autocast(device_type='cuda'):
            age_out, gender_out, race_out = model(images)

            #  losses
            loss_age = criterion_age(age_out, age_labels)
            loss_gender = criterion_gender(gender_out.squeeze(), gender_labels)
            loss_race = criterion_race(race_out, race_labels)

            # coimbine the losses
            loss = alpha * loss_age + beta * loss_gender + gamma * loss_race

        # Backward prop
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

        # get predictions and accuracy
        age_preds = torch.argmax(age_out, dim=1)
        gender_preds = torch.round(torch.sigmoid(gender_out.squeeze()))
        race_preds = torch.argmax(race_out, dim=1)

        correct_age += (age_preds == age_labels).sum().item()
        correct_gender += (gender_preds == gender_labels).sum().item()
        correct_race += (race_preds == race_labels).sum().item()

        total_samples += age_labels.size(0)

        progress_bar.set_postfix({
            "Loss": loss.item(),
            "Age Acc": f"{(correct_age / total_samples) * 100:.2f}%",
            "Gender Acc": f"{(correct_gender / total_samples) * 100:.2f}%",
            "Race Acc": f"{(correct_race / total_samples) * 100:.2f}%"
        })

    #  accuracy
    age_acc = correct_age / total_samples
    gender_acc = correct_gender / total_samples
    race_acc = correct_race / total_samples

    return total_loss / len(dataloader), age_acc, gender_acc, race_acc

def validate(model, dataloader, device):
    model.eval()
    total_loss = 0
    correct_age, correct_gender, correct_race = 0, 0, 0
    total_samples = 0

    progress_bar = tqdm(dataloader, desc="Validating", unit="batch")

    with torch.no_grad(): #no back prop only seeing how the model generalizes
        with torch.amp.autocast(device_type='cuda'):
            for images, age_labels, gender_labels, race_labels in progress_bar:
                images, age_labels, gender_labels, race_labels = (
                    images.to(device),
                    age_labels.to(device),
                    gender_labels.to(device),
                    race_labels.to(device),
                )
                age_out, gender_out, race_out = model(images)

                loss_age = criterion_age(age_out, age_labels)
                loss_gender = criterion_gender(gender_out.squeeze(), gender_labels)
                loss_race = criterion_race(race_out, race_labels)

                loss = loss_age + loss_gender + loss_race
                total_loss += loss.item()

                age_preds = torch.argmax(age_out, dim=1)
                gender_preds = torch.round(torch.sigmoid(gender_out.squeeze()))
                race_preds = torch.argmax(race_out, dim=1)

                correct_age += (age_preds == age_labels).sum().item()
                correct_gender += (gender_preds == gender_labels).sum().item()
                correct_race += (race_preds == race_labels).sum().item()

                total_samples += age_labels.size(0)

                progress_bar.set_postfix({
                    "Loss": loss.item(),
                    "Age Acc": f"{(correct_age / total_samples) * 100:.2f}%",
                    "Gender Acc": f"{(correct_gender / total_samples) * 100:.2f}%",
                    "Race Acc": f"{(correct_race / total_samples) * 100:.2f}%"
                })

    age_acc = correct_age / total_samples
    gender_acc = correct_gender / total_samples
    race_acc = correct_race / total_samples

    return total_loss / len(dataloader), age_acc, gender_acc, race_acc

In [27]:
best_model_path = "best_face_age_gender_race_extraction_state_dict3.pth"

# track the best validation loss
best_val_loss = float("inf")

num_epochs = 10
for epoch in range(num_epochs):
    # training and validation results
    train_loss, train_age_acc, train_gender_acc, train_race_acc = train_one_epoch(model, train_loader, optimizer, device)
    val_loss, val_age_acc, val_gender_acc, val_race_acc = validate(model, val_loader, device)

    #  epoch performance
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Age Acc: {train_age_acc * 100:.2f}%, Gender Acc: {train_gender_acc * 100:.2f}%, Race Acc: {train_race_acc * 100:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Age Acc: {val_age_acc * 100:.2f}%, Gender Acc: {val_gender_acc * 100:.2f}%, Race Acc: {val_race_acc * 100:.2f}%")

    # Check if this is the best 
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)  # Save only state_dict
        print(f"Best model state_dict saved at epoch {epoch+1} with validation loss: {val_loss:.4f}")

Validating: 100%|██████████| 140/140 [00:23<00:00,  5.94batch/s, Loss=3.18, Age Acc=41.73%, Gender Acc=77.27%, Race Acc=45.02%]


Epoch 1/10
Train Loss: 5.1606, Age Acc: 36.86%, Gender Acc: 69.62%, Race Acc: 35.27%
Val Loss: 3.2905, Age Acc: 41.73%, Gender Acc: 77.27%, Race Acc: 45.02%
Best model state_dict saved at epoch 1 with validation loss: 3.2905


Validating: 100%|██████████| 140/140 [00:23<00:00,  5.91batch/s, Loss=2.89, Age Acc=43.88%, Gender Acc=79.23%, Race Acc=49.51%]


Epoch 2/10
Train Loss: 4.7032, Age Acc: 40.77%, Gender Acc: 75.37%, Race Acc: 43.62%
Val Loss: 3.0615, Age Acc: 43.88%, Gender Acc: 79.23%, Race Acc: 49.51%
Best model state_dict saved at epoch 2 with validation loss: 3.0615


Validating: 100%|██████████| 140/140 [00:21<00:00,  6.40batch/s, Loss=2.76, Age Acc=44.91%, Gender Acc=81.53%, Race Acc=51.12%]


...

Epoch 8/10
Train Loss: 4.1816, Age Acc: 44.78%, Gender Acc: 81.37%, Race Acc: 51.01%
Val Loss: 2.6983, Age Acc: 47.47%, Gender Acc: 85.02%, Race Acc: 56.29%
Best model state_dict saved at epoch 8 with validation loss: 2.6983


Validating: 100%|██████████| 140/140 [00:22<00:00,  6.09batch/s, Loss=2.41, Age Acc=47.94%, Gender Acc=85.73%, Race Acc=56.55%]


Epoch 9/10
Train Loss: 4.1440, Age Acc: 45.13%, Gender Acc: 81.72%, Race Acc: 51.73%
Val Loss: 2.6830, Age Acc: 47.94%, Gender Acc: 85.73%, Race Acc: 56.55%
Best model state_dict saved at epoch 9 with validation loss: 2.6830


Validating: 100%|██████████| 140/140 [00:22<00:00,  6.13batch/s, Loss=2.39, Age Acc=48.14%, Gender Acc=86.09%, Race Acc=57.12%]

Epoch 10/10
Train Loss: 4.1072, Age Acc: 45.34%, Gender Acc: 82.08%, Race Acc: 52.31%
Val Loss: 2.6489, Age Acc: 48.14%, Gender Acc: 86.09%, Race Acc: 57.12%
Best model state_dict saved at epoch 10 with validation loss: 2.6489
